<a href="https://colab.research.google.com/github/SarikaKV/Natural-Language-Processing-and-Text-analysis-Projects/blob/master/PayloadEmbeddings_BiLSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os
from os import path
import pandas as pd
import numpy as np
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
from gensim.test.utils import  get_tmpfile
from sklearn.model_selection import KFold, cross_validate, GridSearchCV, cross_val_score, StratifiedKFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import make_scorer, accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

In [0]:
# %tensorflow_version 2.x
# import tensorflow as tf
# device_name = tf.test.gpu_device_name()
# if device_name != '/device:GPU:0':
#   raise SystemError('GPU device not found')
# print('Found GPU at: {}'.format(device_name))

In [0]:
input_path = '/content/drive/My Drive/Tozal/data/'
dataset = 'CSIC_2010_processed' #'ECML_PKDD_2007_processed' #

embedding_dataset = 'CSIC_HTTP_after_sampling.csv' #'ECML_Final_after_sampling.csv' #
#emb_sizes = [5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 60, 70, 80, 90, 100]
emb_size = 10
emb_path = '{}/{}/word2vec_{}.model'.format(input_path, dataset, emb_size)
#is_optimize = True
pad_strategy='post' 
trunc_strategy='post'
max_length = 500

scoring = {'acc': make_scorer(accuracy_score), 'prec': 'precision_micro', 
           'recall': 'recall_micro', 'f1': 'f1_micro'
         }

In [0]:
df = pd.read_csv(os.path.join(input_path, dataset, embedding_dataset)) 
sentences = df.Payload.values
df['len'] =  df['Payload'].str.split(" ").str.len()
print(df['len'].mean())
num_labels = len(set(df['Label'].values))
print(df['Label'].value_counts())

581.0666467185318
1    25065
0    25065
Name: Label, dtype: int64


In [0]:
from keras.preprocessing.sequence import pad_sequences

# integer encode the documents
encoded_docs = [[int(idx) for idx in sentence.split() if idx != '65533'] for sentence in sentences ]

# pad documents to a max length of 4 words
X = pad_sequences(encoded_docs, maxlen=max_length, padding=pad_strategy, truncating=trunc_strategy)


Using TensorFlow backend.


In [0]:
vocab_Set = set()
for sent in encoded_docs:
  for w in sent:
    vocab_Set.add(w)
words2id = dict()
for i in list(vocab_Set):
    words2id[str(i)] = i
vocab_size = len(words2id) + 1
print(words2id)
print("Dataset Vocabulary Size: {}".format(vocab_size))


{'32': 32, '37': 37, '38': 38, '40': 40, '41': 41, '42': 42, '43': 43, '44': 44, '45': 45, '46': 46, '47': 47, '48': 48, '49': 49, '50': 50, '51': 51, '52': 52, '53': 53, '54': 54, '55': 55, '56': 56, '57': 57, '58': 58, '59': 59, '61': 61, '63': 63, '65': 65, '66': 66, '67': 67, '68': 68, '69': 69, '70': 70, '71': 71, '72': 72, '73': 73, '74': 74, '75': 75, '76': 76, '77': 77, '78': 78, '79': 79, '80': 80, '81': 81, '82': 82, '83': 83, '84': 84, '85': 85, '86': 86, '87': 87, '88': 88, '89': 89, '90': 90, '95': 95, '97': 97, '98': 98, '99': 99, '100': 100, '101': 101, '102': 102, '103': 103, '104': 104, '105': 105, '106': 106, '107': 107, '108': 108, '109': 109, '110': 110, '111': 111, '112': 112, '113': 113, '114': 114, '115': 115, '116': 116, '117': 117, '118': 118, '119': 119, '120': 120, '121': 121, '122': 122, '126': 126}
Dataset Vocabulary Size: 80


In [0]:
# get the embedding matrix from the embedding layer
from numpy import zeros
w2v = KeyedVectors.load(emb_path, mmap='r')
#print(w2v['37'])
embedding_matrix = zeros((257, emb_size))
print(embedding_matrix.shape)
for word, i in words2id.items():
  embedding_vector = w2v[str(word)]
  if embedding_vector is not None: # and word != '65533': 
      embedding_matrix[i] = embedding_vector
#print(embedding_matrix[32:])

(257, 10)


/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  import sys


In [0]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
Y_new = df['Label']
Y_new = le.fit_transform(Y_new)

In [0]:
## now splitting into test and training data
from sklearn.model_selection import train_test_split
X_train,X_test, Y_train, Y_test =  train_test_split(X, Y_new,test_size =0.20,random_state= 4 )
print(Y_new)
print(set(df['Label'].values))


[1 1 1 ... 0 0 0]
{0, 1}


In [0]:
# main model
from keras.layers import Input
from keras.models import Model
from keras.layers.embeddings import Embedding
from keras.layers.core import Dense, Reshape, Flatten
from keras.layers import dot, TimeDistributed, Bidirectional, LSTM
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
import time
#print(tf.test.gpu_device_name())
#with tf.device('/device:GPU:0'):
input = Input(shape=(max_length,))
model = Embedding(257,emb_size,weights=[embedding_matrix],input_length=max_length)(input)
model =  Bidirectional (LSTM (100,return_sequences=True,dropout=0.25),merge_mode='concat')(model)
model = TimeDistributed(Dense(100,activation='relu'))(model)
model = Flatten()(model)
model = Dense(100,activation='relu')(model)
if num_labels != 2:
  output = Dense(num_labels,activation='softmax')(model)
  model = Model(input,output)
  model.compile(loss='sparse_categorical_crossentropy',optimizer='adam', metrics=['accuracy'])
else:
  output = Dense(num_labels-1,activation='sigmoid')(model)
  model = Model(input,output)
  model.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy'])
print(model.summary())

earlyStopping = EarlyStopping(monitor='val_loss', patience=3, verbose=0, mode='min')
mcp_save = ModelCheckpoint('{}/{}/best_model_es{}_ml{}.hdf5'.format(input_path, dataset, emb_size, max_length), 
                           save_best_only=True, monitor='val_loss', mode='min', period=1)
reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=7, verbose=1, epsilon=1e-4, mode='min')

start_train = time.time()
history = model.fit(X_train,Y_train,validation_split=0.25, epochs = 20, verbose = 2, callbacks=[earlyStopping, mcp_save,reduce_lr_loss])

print("Model training took {}mins.".format((time.time()-start_train)/60))


Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 500)               0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 500, 10)           2570      
_________________________________________________________________
bidirectional_1 (Bidirection (None, 500, 200)          88800     
_________________________________________________________________
time_distributed_1 (TimeDist (None, 500, 100)          20100     
_________________________________________________________________
flatten_1 (Flatten)          (None, 50000)             0         
_________________________________________________________________
dense_2 (Dense)              (None, 100)               5000100   
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 101 

/usr/local/lib/python3.6/dist-packages/keras/callbacks/callbacks.py:998: UserWarning: `epsilon` argument is deprecated and will be removed, use `min_delta` instead.
  warnings.warn('`epsilon` argument is deprecated and '
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 30078 samples, validate on 10026 samples
Epoch 1/20
 - 790s - loss: 0.3917 - accuracy: 0.7996 - val_loss: 0.2949 - val_accuracy: 0.8469
Epoch 2/20


In [0]:
import matplotlib.pyplot as plt

print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train_accuracy', 'validation_accuracy'], loc='upper left')
plt.savefig('{}/{}/val_acc_es{}_ml{}.png'.format(input_path, dataset, emb_size, max_length))
plt.show()
print('{}/{}/val_acc_es{}_ml{}.png'.format(input_path, dataset, emb_size, max_length))
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train_accuracy', 'validation_accuracy','train_loss', 'validation_loss'], loc='upper left')
plt.savefig('{}/{}/val_loss_es{}_ml{}.png'.format(input_path, dataset, emb_size, max_length))
plt.show()


In [0]:
# evaluate the model
print('{}/{}/best_model_es{}_ml{}.hdf5'.format(input_path, dataset, emb_size, max_length))
model.load_weights('{}/{}/best_model_es{}_ml{}.hdf5'.format(input_path, dataset, emb_size, max_length))
loss, accuracy = model.evaluate(X_test, Y_test, verbose=2)
print('Accuracy: %f' % (accuracy*100))


In [0]:
from sklearn.metrics import classification_report,confusion_matrix
Y_pred = model.predict(X_test)
y_pred = np.array([np.argmax(pred) for pred in Y_pred])
print('  Classification Report:\n',classification_report(Y_test,y_pred),'\n')